<h1 align="center">Computación de Alto Desempeño</h1>
<h1 align="center">OpenMP - Intermedio</h1>
<h1 align="center">2024</h1>
<h1 align="center">MEDELLÍN - COLOMBIA </h1>

***
|[![Outlook](https://img.shields.io/badge/Microsoft_Outlook-0078D4?style=plastic&logo=microsoft-outlook&logoColor=white)](mailto:calvarezh@udemedellin.edu.co)||[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/carlosalvarezh/HPC/blob/main/HPC10_OpenMP_Intermediate.ipynb)
|-:|:-|--:|
|[![LinkedIn](https://img.shields.io/badge/linkedin-%230077B5.svg?style=plastic&logo=linkedin&logoColor=white)](https://www.linkedin.com/in/carlosalvarez5/)|[![@alvarezhenao](https://img.shields.io/twitter/url/https/twitter.com/alvarezhenao.svg?style=social&label=Follow%20%40alvarezhenao)](https://twitter.com/alvarezhenao)|[![@carlosalvarezh](https://img.shields.io/badge/github-%23121011.svg?style=plastic&logo=github&logoColor=white)](https://github.com/carlosalvarezh)|

<table>
 <tr align=left><td><img align=left src="https://github.com/carlosalvarezh/Curso_CEC_EAFIT/blob/main/images/CCLogoColorPop1.gif?raw=true" width="25">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license.(c) Carlos Alberto Alvarez Henao</td>
</table>

***

## **Sincronización en OpenMP**

### **¿Por qué es importante la sincronización?**

Cuando se trabaja con múltiples *threads* en un programa paralelo, el acceso simultáneo a recursos compartidos (como variables globales) puede generar conflictos o resultados inesperados. Por ejemplo, si varios *threads* intentan modificar la misma variable al mismo tiempo, se puede generar una condición de carrera (más sobre esto adelante). La sincronización se utiliza para evitar estos problemas y garantizar que los *threads* trabajen de manera coordinada y ordenada.

En OpenMP, hay varias herramientas para manejar la sincronización, dependiendo de lo que necesites hacer:

### **Secciones Críticas (`#pragma omp critical`)**

Cuando varios *threads* necesitan modificar una misma variable o realizar una tarea que no puede hacerse en paralelo, se utiliza una **sección crítica**. Esta asegura que solo un *thread* a la vez ejecute esa porción de código.

**Ejemplo:** Modificación de una variable compartida

In [ ]:
#include <stdio.h>
#include <omp.h>

int main() {
    int contador = 0;

    #pragma omp parallel for
    for (int i = 0; i < 1000; i++) {
        #pragma omp critical
        {
            contador++;
        }
    }

    printf("El valor final del contador es: %d\n", contador);
    return 0;
}

En este ejemplo, el acceso a la variable `contador` está protegido por `#pragma omp critical`, lo que garantiza que solo un *thread* a la vez incremente su valor. Sin la sección crítica, los resultados serían inconsistentes.

### **Sección Atómica (`#pragma omp atomic`)**

Para operaciones sencillas como sumas o restas, una sección crítica puede ser un poco "costosa" en términos de rendimiento. Aquí es donde la **sección atómica** se convierte en una opción mejor, ya que es más eficiente y asegura que operaciones simples como `x += 1` se ejecuten de manera atómica, es decir, sin interferencia de otros *threads*.

**Ejemplo:** Incremento eficiente de una variable compartida

In [ ]:
#include <stdio.h>
#include <omp.h>

int main() {
    int suma = 0;

    #pragma omp parallel for
    for (int i = 0; i < 1000; i++) {
        #pragma omp atomic
        suma += 1;
    }

    printf("El valor final de la suma es: %d\n", suma);
    return 0;
}

Aquí usamos `#pragma omp atomic` en lugar de `critical`. Esta es una opción más eficiente cuando lo único que se necesita es garantizar que una operación simple, como un incremento, se ejecute sin interferencias.

### **Barrera (`#pragma omp barrier`)**

Imagina un grupo de *threads* corriendo a lo largo de un programa. Cada uno ejecuta su parte del trabajo a su propio ritmo. Pero, ¿qué sucede si quieres asegurarte de que todos los *threads* terminen una tarea antes de que el programa continúe? Aquí es donde entra la **barrera**.

La barrera obliga a que todos los *threads* lleguen a cierto punto en el programa antes de que cualquiera de ellos pueda continuar. Es una forma de asegurarse de que todo el mundo esté sincronizado antes de pasar al siguiente paso.

**Ejemplo:** Coordinación de *threads* con una barrera

In [ ]:
#include <stdio.h>
#include <omp.h>

int main() {
    #pragma omp parallel
    {
        int id = omp_get_thread_num();
        printf("Thread %d: Antes de la barrera\n", id);

        #pragma omp barrier  // Todos los threads deben llegar aquí antes de continuar

        printf("Thread %d: Después de la barrera\n", id);
    }
    return 0;
}

En este ejemplo, los *threads* imprimen un mensaje antes y después de la barrera. Ningún *thread* puede continuar más allá de la barrera hasta que todos los *threads* hayan llegado a ese punto.

### **Ordered (`#pragma omp ordered`)**

A veces, incluso cuando ejecutas un bucle en paralelo, puede que quieras que ciertos resultados aparezcan en el mismo orden en que habrían aparecido si el bucle se ejecutara de forma secuencial. Para estos casos, puedes usar la directiva **`#pragma omp ordered`**.

**Ejemplo:** Garantizar el orden en la salida

In [ ]:
#include <stdio.h>
#include <omp.h>

int main() {
    #pragma omp parallel for ordered
    for (int i = 0; i < 10; i++) {
        #pragma omp ordered
        {
            printf("Iteración %d\n", i);
        }
    }
    return 0;
}

Aunque el bucle se ejecuta en paralelo, gracias a `#pragma omp ordered`, la salida del programa sigue apareciendo en orden secuencial. Esta directiva es útil cuando el orden de ejecución es importante.

## **Tareas (Tasking) en OpenMP**

### **¿Qué es una tarea?**

En algunos casos, el paralelismo en bucles no es suficiente. Necesitamos dividir el trabajo en **tareas**, que son bloques de código independientes que pueden ejecutarse en paralelo. OpenMP ofrece la capacidad de crear tareas que son distribuidas dinámicamente entre los *threads* disponibles.

### **Creación de Tareas**

Las tareas en OpenMP son bloques de trabajo que pueden ser creados de forma dinámica y ejecutados por cualquier *thread*. Esto permite mucha flexibilidad, especialmente cuando las iteraciones de un bucle no son homogéneas, es decir, cuando algunas tareas pueden tomar más tiempo que otras.

**Ejemplo:** Creación y ejecución de tareas

In [ ]:
#include <stdio.h>
#include <omp.h>

void tarea(int id) {
    printf("Ejecutando tarea %d\n", id);
}

int main() {
    #pragma omp parallel
    {
        #pragma omp single
        {
            for (int i = 0; i < 10; i++) {
                #pragma omp task
                tarea(i);
            }
        }
    }
    return 0;
}

En este ejemplo, un solo *thread* crea las tareas (`#pragma omp single`), pero cualquier *thread* disponible puede ejecutar esas tareas. Esto es útil cuando las tareas son independientes y no tienen que ejecutarse en ningún orden particular.

### **Tareas con dependencias**

En algunos casos, puede que una tarea dependa del resultado de otra. Por ejemplo, no puedes procesar un archivo hasta que haya sido descargado. Para estos casos, OpenMP permite definir dependencias entre tareas.

**Ejemplo:** Tareas con dependencias

In [ ]:
#include <stdio.h>
#include <omp.h>

int main() {
    int x = 0;

    #pragma omp parallel
    {
        #pragma omp single
        {
            // Tarea 1: Modifica el valor de x
            #pragma omp task depend(out:x)
            {
                x = 10;
                printf("Tarea 1: x = %d\n", x);
            }

            // Tarea 2: Depende del valor de x
            #pragma omp task depend(in:x)
            {
                printf("Tarea 2 (después de Tarea 1): x = %d\n", x);
            }
        }
    }
    return 0;
}

Aquí, la segunda tarea se ejecuta solo después de que la primera haya completado su operación sobre `x`, gracias a las dependencias especificadas con `depend`.

## **Condiciones de Carrera y Bloqueos**

### **¿Qué es una Condición de Carrera?**

Una **condición de carrera** ocurre cuando dos o más *threads* acceden a una variable compartida al mismo tiempo y al menos uno de ellos la modifica. Esto puede llevar a resultados impredecibles, ya que no hay garantías de en qué orden los *threads* accederán a la variable.

**Ejemplo:** Condición de carrera

In [ ]:
#include <stdio.h>
#include <omp.h>

int main() {
    int x = 0;

    #pragma omp parallel for
    for (int i = 0; i < 1000; i++) {
        x += 1;  // Condición de carrera potencial
    }

    printf("Valor de x: %d\n", x);
    return 0;
}

Aquí, varios *threads* están modificando la variable `x` al mismo tiempo, lo que provoca un resultado incorrecto debido a la condición de carrera. Dependiendo de la máquina y el entorno de ejecución, el valor final de `x` será incorrecto.

### **Solución: Sección Crítica o Atómica**

Para evitar condiciones de carrera, necesitamos asegurarnos de que solo un *thread* a la vez modifique la variable compartida.

**Ejemplo:** Solución con sección atómica

In [ ]:
#include <stdio.h>
#include <omp.h>

int main() {
    int x = 0;



    #pragma omp parallel for
    for (int i = 0; i < 1000; i++) {
        #pragma omp atomic
        x += 1;  // Actualización atómica
    }

    printf("Valor de x: %d\n", x);
    return 0;
}

Con `#pragma omp atomic`, evitamos la condición de carrera y garantizamos que el valor de `x` se actualice correctamente sin interferencias de otros *threads*.

## **Paralelismo SIMD**

### **¿Qué es el Paralelismo SIMD?**

SIMD (Single Instruction, Multiple Data) es una técnica que permite ejecutar una única instrucción en múltiples datos al mismo tiempo. Es especialmente útil para trabajar con vectores o arreglos grandes, y puede acelerar significativamente ciertos cálculos.

### **Vectorización con OpenMP**

OpenMP permite vectorizar bucles con la directiva `#pragma omp simd`, forzando al compilador a ejecutar las operaciones de manera paralela en múltiples elementos del arreglo.

**Ejemplo:** Paralelismo SIMD

In [ ]:
#include <stdio.h>
#include <omp.h>

int main() {
    int n = 100;
    float a[n], b[n];

    // Inicialización de los arreglos
    for (int i = 0; i < n; i++) {
        a[i] = i * 1.0f;
    }

    // Paralelismo SIMD: Multiplicar todos los elementos del arreglo
    #pragma omp simd
    for (int i = 0; i < n; i++) {
        b[i] = a[i] * 2.0f;
    }

    // Mostrar algunos resultados
    for (int i = 0; i < 10; i++) {
        printf("b[%d] = %f\n", i, b[i]);
    }

    return 0;
}

Aquí estamos utilizando `#pragma omp simd` para paralelizar el bucle de multiplicación, haciendo que se procesen varios elementos del arreglo `a[]` en paralelo.

## **Ejercicios**

**Ejercicio 1: Suma de series paralelas con condiciones de carrera -** Dado un vector de números aleatorios, paraleliza el cálculo de la suma de sus elementos utilizando OpenMP. Inicialmente, introduce una condición de carrera para que los estudiantes vean el problema. Luego, deben solucionarlo utilizando una sección crítica o una directiva atómica.

```pseudocode
- Generar un vector aleatorio de 1,000,000 números.
- Calcular la suma de los elementos en paralelo.
- Corregir la condición de carrera usando `#pragma omp critical` o `#pragma omp atomic`.
```

**Desafío adicional**: Extiende el programa para que también calcule la suma cuadrada (sumar el cuadrado de cada número del vector) en paralelo, garantizando la sincronización adecuada.


**Ejercicio 2: Paralelización del cálculo de la norma de un vector** - Implementar paralelismo en el cálculo de la **norma de un vector** en un espacio N-dimensional, aplicando la directiva `#pragma omp simd` para optimizar el cálculo.

La norma de un vector $\vec{v}$ de dimensión N se calcula como:

$$
\|\vec{v}\| = \sqrt{v_1^2 + v_2^2 + \dots + v_n^2}
$$

Escribe un programa en C que:
1. Genere un vector con N componentes aleatorias.
2. Calcule la norma utilizando paralelismo SIMD para los cuadrados de los elementos y sincronización para la suma total.

```c
// Pseudocódigo
- Inicializar un vector con números aleatorios.
- Paralelizar el cálculo de la suma de los cuadrados utilizando `#pragma omp simd`.
- Sincronizar los resultados y calcular la raíz cuadrada.
```

**Desafío adicional**: Aumentar el tamaño del vector a 10 millones de elementos y observar el impacto en el tiempo de ejecución antes y después de la vectorización.

**Ejercicio 3: Movimiento rectilíneo uniformemente acelerado (MRUA) -** Modela el movimiento de un objeto bajo **movimiento rectilíneo uniformemente acelerado (MRUA)**, donde el objeto parte de una posición inicial con una velocidad inicial, y se actualizan la posición y la velocidad en función de la aceleración $a$. Utiliza OpenMP para calcular las posiciones y velocidades en varios puntos de tiempo de forma paralela, pero respetando las dependencias (cada posición futura depende de la posición anterior).

Las fórmulas para MRUA son:
$$
v(t) = v_0 + at
$$
$$
x(t) = x_0 + v_0 t + \frac{1}{2} a t^2
$$

```c
// Pseudocódigo
- Inicializar valores de velocidad y posición.
- Crear tareas para calcular posición y velocidad en diferentes tiempos t.
- Utilizar dependencias para asegurar que cada tarea respete los cálculos previos.
```

**Desafío adicional**: Agrega un factor de fricción al sistema y paraleliza tanto el cálculo de la velocidad como de la posición.


**Ejercicio 4: Multiplicación de matrices grandes -** Implementar la multiplicación de dos matrices grandes utilizando el paralelismo en bucles de OpenMP. Este ejercicio ayudará a reforzar la idea de cómo se pueden paralelizar las operaciones de matrices, optimizando el tiempo de ejecución en grandes dimensiones.

Escribe un programa en C que realice la multiplicación de dos matrices \(A \times B = C\) de tamaño \(N \times N\). Paraleliza el cálculo de los elementos de la matriz resultante \(C\).

```c
// Pseudocódigo
- Inicializar matrices A y B de tamaño N x N.
- Implementar la multiplicación de matrices tradicional.
- Paralelizar los bucles utilizando `#pragma omp parallel for`.
- Comparar el rendimiento para N = 500, 1000 y 2000.
```

**Desafío adicional**: Aplicar optimización con `#pragma omp collapse` para paralelizar bucles anidados.


**Ejercicio 5: Cálculo de Pi usando el método Monte Carlo -** Este método consiste en generar puntos aleatorios dentro de un cuadrado y contar cuántos caen dentro de un círculo inscrito. La razón entre los puntos dentro del círculo y el total de puntos generados aproxima el valor de $\pi$.

Escribe un programa que:
1. Genere puntos aleatorios.
2. Determine cuántos de ellos caen dentro del círculo.
3. Calcule $\pi$ en paralelo.

```c
// Pseudocódigo
- Generar puntos aleatorios en el plano (x, y).
- Contar los puntos que caen dentro del círculo (x^2 + y^2 <= 1).
- Paralelizar el proceso de conteo utilizando `#pragma omp parallel for`.
```

**Desafío adicional**: Comparar la eficiencia de una implementación que utilice reducción (`reduction`) frente a una implementación con tareas (`task`).


**Ejercicio 6: Paralelismo SIMD en el cálculo de producto punto -** Utilizar la directiva `#pragma omp simd` para optimizar el cálculo del **producto punto** de dos vectores grandes. Escribe un programa que paralelice el cálculo del producto punto entre dos vectores de tamaño 10 millones de elementos.

```c
// Pseudocódigo
- Inicializar dos vectores con números aleatorios.
- Implementar el cálculo del producto punto en un bucle simple.
- Utilizar `#pragma omp simd` para vectorizar el cálculo.
```

**Desafío adicional**: Implementa la misma operación utilizando `reduction` y compara la eficiencia de ambas versiones.


**Ejercicio 7: Resolución de un sistema de ecuaciones lineales con Gauss-Seidel -** Implementar el método de Gauss-Seidel para resolver un sistema de ecuaciones lineales y paralelizar las iteraciones. Escribe un programa que utilice el método iterativo de Gauss-Seidel para resolver un sistema de ecuaciones de la forma $Ax = b$.

```c
// Pseudocódigo
- Inicializar una matriz A y un vector b.
- Implementar el método de Gauss-Seidel.
- Paralelizar el cálculo utilizando OpenMP.
```

**Desafío adicional**: Usar la directiva `#pragma omp task` para paralelizar las actualizaciones de cada fila en cada iteración.
